In [ ]:
import nest_asyncio

nest_asyncio.apply()

import asyncio
import httpx

async def main():
    url = "http://127.0.0.1:8000/chat/stream"
    json_data = {
        "model": "GLM-4.5-Flash",
        "message": "你可以做什么"
    }

    async with httpx.AsyncClient(timeout=None) as client:
        async with client.stream("POST", url, json=json_data) as response:
            async for chunk in response.aiter_text():
                print(chunk, end="", flush=True)

asyncio.get_event_loop().run_until_complete(main())


In [ ]:
import requests

url = "http://124.222.61.180:8000/chat"
# url="http://127.0.0.1:8000/chat"
payload = {
    "model": "GLM-4.5-Flash",
    "message": "请帮我写一个python请求示例"
}

response = requests.post(url, json=payload)
data = response.json()

print("回复内容:", data.get("reply"))


In [10]:
import requests
import json

url = "http://124.222.61.180:8000/chat/mcp/stream"
url="http://127.0.0.1:8000/chat/mcp/stream"

payload = {
    "model": "GLM-4.5-Flash",
    "message": "今天上海天气"
}

# 流式请求
with requests.post(url, json=payload, stream=True) as response:
    for line in response.iter_lines(decode_unicode=True):
        if line:
            try:
                # 如果服务端每行是 JSON，可以解析
                data = json.loads(line)
                print("流式输出:", data.get("reply", data))
            except json.JSONDecodeError:
                # 如果服务端每行是纯文本
                print(line)


根据最新数据，上海今天的天气情况如下：当前温度为30.2°C，天气晴朗，湿度为75%。东南风风速为16.9公里/小时，气压1012毫巴，无降水。体感温度较高，达到37.8°C。目前是夜晚，能见度良好，为10公里。


In [ ]:
import requests
import os
import openai

SERVER_IP = os.environ.get("SERVER_IP", "")
response = requests.get("http://124.222.61.180:8000/monitor")
response = requests.get("http://127.0.0.1:8000/monitor")
if response.status_code == 200:
    data = response.json()
    print("监控数据:", data)
else:
    print("请求失败，状态码:", response.status_code)


In [ ]:

from app.api.mcp.mcp_tools.register_tool import parse_curl_and_register,tools
curl_text = 'curl -X GET "https://api.weatherapi.com/v1/current.json?key=c502ec55fecd4b0d969141024251008&q=hefei&lang=zh" -H "accept: application/json"'
parse_curl_and_register(curl_text, "天气", auth_required=False)

# 调用并覆盖查询参数 q
result = tools["天气"]["func"](params={"q": "shanghai"})
print(result)


In [ ]:
print(tools)

In [ ]:
# 调用工具
result = tools["天气"]["func"]()
print(result)

In [ ]:
from api.mcp.mcp_tools.tools import save_tool_to_db, load_tool_from_db
from api.mcp.mcp_tools.register_tool import tools
save_tool_to_db(
    name="天气查询",
    description="根据地点查询天气",
    curl_cmd='curl -X GET "https://api.weatherapi.com/v1/current.json?key=c502ec55fecd4b0d969141024251008&q=hefei&lang=zh" -H "accept: application/json"',
    auth_required=False,
    modifiable_fields={"q": "城市名称"},  # dict 格式
    usage_example={"q": "Hefei"}         # dict 格式
)
load_tool_from_db("天气查询")
print(tools)

In [ ]:
result = tools["天气查询"]["func"](params={"q": "shanghai"})

In [ ]:
print(result)

In [3]:
import requests
import json
from api.prompts.mcp_prompts import QUERY_BOOL
query_bool_prompt = QUERY_BOOL.format(question="今天上海天气怎么样")
url = "http://124.222.61.180:8000/chat/mcp/stream"
url="http://127.0.0.1:8000/chat/mcp/stream"

payload = {
    "model": "GLM-4.5-Flash",
    "message": "今天上海天气怎么样"
}

# 流式请求
with requests.post(url, json=payload, stream=True) as response:
    for line in response.iter_lines(decode_unicode=True):
        if line:
            try:
                # 如果服务端每行是 JSON，可以解析
                data = json.loads(line)
                print("流式输出:", data.get("reply", data))
            except json.JSONDecodeError:
                # 如果服务端每行是纯文本
                print(line)


Internal Server Error


In [1]:
from entension.my_openai import MyOpenAI as OpenAI
import os
from api.chat.chatbot.multi_tool_agent import check_answer_direct, select_tool, fill_tool_params
request_message="今天天气上海怎么样"
SERVER_IP = os.environ.get("SERVER_IP", "")
client = OpenAI(api_key=os.environ.get("PASSWORD", ""),
                base_url=f"http://{SERVER_IP}:8000/v1")
query_bool=check_answer_direct(client, "GLM-4.5-Flash", request_message)

In [2]:
print(query_bool)

需要工具


In [ ]:
from api.mcp.mcp_tools.tools import save_tool_to_db, load_tool_from_db,get_tool_details
from api.mcp.mcp_tools.register_tool import tools
load_tool_from_db("天气查询")
print(tools)

[REGISTER] Tool '天气查询' registered successfully!
{'天气查询': {'func': <function parse_curl_and_register.<locals>.tool_func at 0x000001D516EB24D0>, 'auth_required': False}}


In [4]:
from api.mcp.mcp_tools.tools import get_tools_dict
tools_list = get_tools_dict()
seltc =select_tool(client, "GLM-4.5-Flash", request_message, tools_list)

In [5]:
tools_list

{'天气查询': '根据地点查询天气'}

In [6]:
tools_list[seltc]

'根据地点查询天气'

In [7]:
details = get_tool_details(seltc)
details["modifiable_fields"]
jsom=fill_tool_params(client,"GLM-4.5-Flash",request_message,seltc,tools_list[seltc],details)

In [8]:
jsom

'{"q": "上海"}'

In [10]:
import json

In [11]:
obj = json.loads(jsom)

In [12]:
tools["天气查询"]["func"](params=obj)

{'location': {'name': '上海',
  'region': 'Shanghai',
  'country': '中国',
  'lat': 31.005,
  'lon': 121.4086,
  'tz_id': 'Asia/Shanghai',
  'localtime_epoch': 1755351077,
  'localtime': '2025-08-16 21:31'},
 'current': {'last_updated_epoch': 1755351000,
  'last_updated': '2025-08-16 21:30',
  'temp_c': 30.3,
  'temp_f': 86.5,
  'is_day': 0,
  'condition': {'text': '晴朗',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 10.7,
  'wind_kph': 17.3,
  'wind_degree': 135,
  'wind_dir': 'SE',
  'pressure_mb': 1012.0,
  'pressure_in': 29.88,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 75,
  'cloud': 0,
  'feelslike_c': 37.9,
  'feelslike_f': 100.3,
  'windchill_c': 27.9,
  'windchill_f': 82.1,
  'heatindex_c': 31.9,
  'heatindex_f': 89.5,
  'dewpoint_c': 24.0,
  'dewpoint_f': 75.1,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 0.0,
  'gust_mph': 16.1,
  'gust_kph': 25.8,
  'short_rad': 0,
  'diff_rad': 0,
  'dni': 0,
  'gti': 0}}

In [13]:
import requests

url = "http://124.222.61.180:8000/tools"  # 你的 FastAPI 默认本地地址
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("工具列表：", data)
else:
    print("请求失败", response.status_code, response.text)


工具列表： {'tools': {'天气查询': '根据地点查询天气'}}


In [10]:
import requests
import json

url = "http://124.222.61.180:8000/tools/register"

# 构造请求 payload
payload = {
    "name": "天气查询abv",
    "description": "根据地点查询天气",
    "curl": "curl https://api.weatherapi.com/v1/current.json?q=Hefei",
    "auth_required": True,
    "modifiable_fields": {"q": "城市名称"},
    "usage_example": {"q": "Hefei"}
}

# 发送 POST 请求
response = requests.post(url, data=json.dumps(payload), headers={"Content-Type": "application/json"})

# 查看返回结果
if response.status_code == 200:
    print("注册成功:", response.json())
else:
    print("注册失败:", response.status_code, response.text)


注册成功: {'message': '工具 天气查询abv 注册成功'}


In [12]:
import requests

tool_name = "天气查询abv"
url = f"http://124.222.61.180:8000/tools/{tool_name}"

response = requests.delete(url)

if response.status_code == 200:
    print("删除成功:", response.json())
else:
    print("删除失败:", response.status_code, response.text)


删除成功: {'message': "工具 '天气查询abv' 已删除"}
